<a href="https://colab.research.google.com/github/chloevan/gcp_tutorial/blob/master/settings/Ch03_bigquery_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: "Ch03_bigquery_with_python"
date: 2020-04-16T11:40:30+09:00
tags:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "빅쿼리 파이썬"
  - "BigQuery with Python"
categories:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "Python"
menu: 
  gcp:
    name: Ch03_bigquery_with_python
---

## 1. 구글 클라우드 설정

본격적인 빅쿼리 실습에 앞서서, Python과 연동하는 예제를 준비하였다. 빅쿼리 시작에 앞서서 선행적으로 클라우드 사용을 해야 한다. 

1. 만약 GCP 프로젝트가 없다면, 계정을 연동한다. Go to [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager)
2. 그리고, 비용결제를 위한 카드를 등록한다. [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) 
3. 마지막으로 BigQuery API를 사용해야 하기 때문에 빅쿼리 API 사용허가를 내준다.[Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery)

위 API를 이용하지 않으면 `Python` 또는 `R`과 연동해서 사용할 수는 없다. 자주 쓰는것이 아니라면 비용은 거의 발생하지 않으니 염려하지 않아도 된다. 비용관리에 대한 자세한 내용은 [BigQuery 권장사항: 비용 관리](https://cloud.google.com/bigquery/docs/best-practices-costs?hl=ko)에서 확인하기를 바란다. 

## 2. 사용자 계정 인증
구글 코랩을 사용해서 인증 절차를 밟도록 한다. 아래 소스코드는 변경시키지 않는다. 아래 절차대로 진행하면 된다. `Gmail` 인증 절차와 비슷하다. 


In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## 3. 매직(magics)을 활용한 BigQuery 연동

아래 `%%` 코드를 활용하면 빅쿼리를 이제 `Jupyter Notebook`에서 사용할 수 있다. 이제 강사가 보여주는 모든 코드는 빅쿼리 GUI에서 확인하는 것이 아닌 `Jupyter Notebook`에서 사용할 것이다. 

```python
%%bigquery --project yourprojectid
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`
```

In [11]:
%%bigquery --project bigquerytutorial-274406
SELECT
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


## 4. BigQuery를 SQL처럼 연동하기 (google.cloud 모듈 활용)

연동 방법에 관한 자세한 내용은 `BigQuery` 공식 문서를 활용해보자. See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

In [0]:
project_id = 'bigquerytutorial-274406'

In [15]:
from google.cloud import bigquery
from tabulate import tabulate

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


In [17]:
print(tabulate(df.describe(), tablefmt="pipe", headers="keys"))

|       |   station_number |   wban_number |      year |      month |        day |   mean_temp |   num_mean_temp_samples |   mean_dew_point |   num_mean_dew_point_samples |   mean_sealevel_pressure |   num_mean_sealevel_pressure_samples |   mean_station_pressure |   num_mean_station_pressure_samples |   mean_visibility |   num_mean_visibility_samples |   mean_wind_speed |   num_mean_wind_speed_samples |   max_sustained_wind_speed |   max_gust_wind_speed |   max_temperature |   total_precipitation |   snow_depth |
|:------|-----------------:|--------------:|----------:|-----------:|-----------:|------------:|------------------------:|-----------------:|-----------------------------:|-------------------------:|-------------------------------------:|------------------------:|------------------------------------:|------------------:|------------------------------:|------------------:|------------------------------:|---------------------------:|----------------------:|------------------:|--

여기에서 흥미로운 것이 있다면, `BigQuery`에서 불러온 데이터와 `pandas` 문법 연동이 가능하다는 것이다. 위 소스코드를 천천히 읽어보기를 권한다. 

## 5. pandas-gbq 모듈을 활용한 BigQuery 사용

`Pandas` `gbq` 라이브러리는 `pandas` 공동체가 주도하는 프로젝트다. `DataFrame을 BigQuery`에 쓰고 쿼리를 실행하는 것과 같은 기본적인 기능을 다룬다. 자세한 건, 모듈의 공식문서를 확인해본다. [Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [19]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

print(tabulate(df.head(), tablefmt="pipe", headers="keys"))

|    | name    |   count |
|---:|:--------|--------:|
|  0 | James   |  272793 |
|  1 | John    |  235139 |
|  2 | Michael |  225320 |
|  3 | Robert  |  220399 |
|  4 | David   |  219028 |


이 글을 읽음으로써, 이제 매우 쉽게 빅쿼리와 연동해서 사용할 수 있게 되었다. 다음에는 `R`과 연동해서 사용하는 방법에 대해 익히도록 한다. 

## 6. Reference
“Getting Started with BigQuery.” Google, Google, colab.research.google.com/notebooks/bigquery.ipynb.